In [95]:
# Importing the necessary libraries
import torch
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import pandas as pd
import numpy as np

# Load the numerical data you want to train BERT on
data = pd.read_csv('data.csv')
data.dropna(inplace=True)

# Convert numerical values to string format to match BERT input requirement
data = data.astype(str)

data['sequence'] = ''

# Concatenate all the values in a row into a single string using the column names
# Iterate through rows and columns
for index, row in data.iterrows():
    string = ''
    for column in data.columns:
        if column != 'sequence':
            # Concatenate column name with row value
            string += column + ': ' + str(row[column]) + ' '
    data['sequence'][index] = string

data.describe


<bound method NDFrame.describe of           id       author geometry pressure [MPa] mass_flux [kg/m2-s]  \
3          3         Beus  annulus          13.79              3679.0   
9          9       Peskov     tube           12.0              1617.0   
15        15     Thompson     tube          12.07              3255.0   
17        17     Thompson     tube          10.41              2563.0   
30        30      Janssen  annulus           9.68              5615.0   
...      ...          ...      ...            ...                 ...   
31623  31623      Janssen  annulus           8.27              2731.0   
31624  31624  Weatherhead     tube          13.79               963.0   
31626  31626     Thompson     tube          15.51              2984.0   
31627  31627     Thompson     tube           0.64              3282.0   
31635  31635     Thompson     tube          17.24              2984.0   

      x_e_out [-] D_e [mm] D_h [mm] length [mm] chf_exp [MW/m2]  \
3         -0.0279     

In [73]:
# Define the BERT classifier to be used to train the model
class BERTClassifier(torch.nn.Module):
    def __init__(self, dropout_rate=0.3):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.dropout = torch.nn.Dropout(dropout_rate)
        self.linear = torch.nn.Linear(768, 1)

    def forward(self, input_ids, attention_mask):
        _, pooled_out = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        dropout_out = self.dropout(pooled_out)
        linear_out = self.linear(dropout_out)
        return linear_out

In [103]:
# Load the pretrained BERT model and tokenizer to convert data to tokenize-able format
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

MAX_LENGTH = 64
# Use the tokenizer to convert the data into tokens and then into PyTorch tensor format
# Get the maximum length of all the sentences to pad the shorter ones to match that format
tokenized_data = data['sequence'].apply((lambda x: tokenizer.encode_plus(x, 
                                                                    add_special_tokens=True,
                                                                    padding='longest')
                                        )
                                        )

#a = len(data['sequence'])
tokenized_data = tokenized_data.reset_index(drop=True)
input_data = torch.tensor(tokenized_data)


# Define the optimizer to be used to train the model
dropout_rate = 0.3
model = BERTClassifier(dropout_rate=dropout_rate)
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

# Train the model over a set number of epochs
epochs = 4
for epoch in range(epochs):
    epoch_loss = 0
    for i in range(len(input_data)):
        # Reset gradients
        model.zero_grad()
        
        # Forward pass
        input_ids = input_data[i]
        attention_mask = [int(token_id > 0) for token_id in input_ids]
        attention_mask = torch.tensor(attention_mask)
        y_pred = model.forward(input_ids.view(1,-1), attention_mask.view(1,-1))
        
        # Compute loss
        y_true = torch.tensor([float(labels[i])])
        loss_func = torch.nn.BCEWithLogitsLoss()
        loss = loss_func(y_pred.view(1,-1), y_true.view(1,-1))
        epoch_loss += loss.item()

        # Backward pass
        loss.backward()
        
        # Update weights
        optimizer.step()
        
    # Print loss per epoch 
    print(f'Epoch: {epoch+1}, Loss: {epoch_loss}')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ValueError: could not determine the shape of object type 'BatchEncoding'

In [63]:
a = ([tokenized_data])
a = np.transpose(a)

In [90]:
a = len(data['sequence'])
b = data['sequence'].values

In [102]:
len(tokenized_data[100])

92